In [77]:
import numpy as np
import pandas as pd
import sympy
from ast import literal_eval
import os, time

# Random Functions

In [78]:
RESULT_DIR = "final_results"
benchmark = "rational_2"

In [79]:
df = {}
functions = None
for f_name in os.listdir(RESULT_DIR):
    try:
        benchmark_name = None
        if len(f_name.split("_")) == 4:
            benchmark_name = f_name.split("_")[1]
        elif len(f_name.split("_")) == 5:
            benchmark_name = "_".join(f_name.split("_")[1:-2])
                
        tool_name =  f_name.split("_")[-2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
            
            if tool_name == 'volesti':
                try:
                    df['volesti']['details'] = df['volesti']['details'].apply(literal_eval)
                                        
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['output'] for i in x]))
                    df['volesti']['output'] = list(zip(df['volesti']['np'].apply(np.mean), df['volesti']['np'].apply(np.std)))            
                
                    df['volesti']['np'] = df['volesti']['details'].apply(lambda x: np.asarray([i['time'] for i in x]))
                    df['volesti']['time'] = df['volesti']['np'].apply(np.mean)            
                except:
                    pass           
                 
            if tool_name == 'faza':
                # try:
                df['faza']['details'] = df['faza']['details'].apply(literal_eval)
                df['faza']['n_hrects'] = df['faza']['details'].apply(lambda x: x['logs'][0]['hrect_checked_num'] if 'logs' in x else x['output'][-1]['hrect_checked_num'])
                # except:
                #     pass

    except Exception as e:
        print(e)

list index out of range


In [81]:
tools = sorted(list(df.keys()))
tool0 = tools[0]
tool1 = tools[1]

df_res = df[tool0].join(df[tool1], on='index', lsuffix=f'_{tool0}', rsuffix=f'_{tool1}', how='outer')

for t in tools[2:]:
    df_res = df_res.join(df[t].add_suffix(f'_{t}'), on='index', how='outer')


In [83]:
df_res.columns

Index(['index', 'bechmark_faza', 'formula_faza', 'index_faza', 'output_faza',
       'error_faza', 'time_faza', 'details_faza', 'n_hrects', 'bechmark_gubpi',
       'formula_gubpi', 'bounds', 'index_gubpi', 'output_gubpi', 'error_gubpi',
       'time_gubpi', 'details_gubpi', 'bechmark_latte', 'formula_latte',
       'bounds_latte', 'index_latte', 'output_latte', 'error_latte',
       'time_latte', 'details_latte', 'bechmark_volesti', 'formula_volesti',
       'bounds_volesti', 'index_volesti', 'output_volesti', 'error_volesti',
       'time_volesti', 'details_volesti', 'np_volesti'],
      dtype='object')

In [ ]:
df_res = df_res.sort_values('index_gubpi')
selected_cols = [f'formula_gubpi']
selected_cols.extend(['bounds'])
# selected_cols.extend([f'bounds_{t}' for t in tools])


for t in tools:
    selected_cols.append(f"output_{t}")
    df_res[f"output_{t}"] = df_res[f"output_{t}"].fillna(df_res[f"error_{t}"]).fillna('TBD').astype(str).str[:1000]
    selected_cols.append(f"time_{t}")
    df_res[f"time_{t}"] = df_res[f"time_{t}"].fillna(0).apply(lambda x:round(x, 3)) 
    selected_cols.append(f"n_hrects")
    
df_res = df_res[selected_cols]
df_res['formula_gubpi'] = df_res['formula_gubpi'].apply(sympy.parse_expr)
df_res

KeyError: "['bounds_gubpi'] not in index"

In [ ]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)